# Sentiment analysis for stock price prediction


#### Introduction

This notebook ...

#### Grab data sources and merge into a single big dataframe

In [12]:
# Ensure correct packages and settings
import os
import sys
import pathlib
import pandas as pd
from pathlib import Path

# Check if running within a virtual environment
if sys.prefix != sys.base_prefix:
    print(f"Running in a virtual environment: {sys.prefix}")
else:
    print("Not running in a virtual environment. Activate the environment, install the packages and try again.")
    sys.exit(1)

Running in a virtual environment: c:\mainrism\imp\enterprises\projects\tweet-stock-sentiment\.venv


In [10]:
# Import NVDA daily stock prices CSV into a DataFrame
nvda_df = pd.read_csv(Path("data/nvda-daily-stock-prices.csv"))
nvda_df.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,1999-01-22,0.037615,0.041016,0.048828,0.038802,0.043750,2714688000
1,1999-01-25,0.041556,0.045313,0.045833,0.041016,0.044271,510480000
2,1999-01-26,0.038331,0.041797,0.046745,0.041146,0.045833,343200000
3,1999-01-27,0.038212,0.041667,0.042969,0.039583,0.041927,244368000
4,1999-01-28,0.038092,0.041536,0.041927,0.041276,0.041667,227520000
